In [1]:
import random
import torch,torchvision
import cv2
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import wandb
import numpy as np
from torch.nn import *
from torch.optim import *
from torchvision.transforms import *
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [2]:
IMG_SIZE = 224
PROJECT_NAME = 'Fake-Face-V2'
device = torch.device('cuda')

In [3]:
transformation = Compose([
    ToTensor(),
])

In [4]:
def load_data(data_dir='./data/',transformation=transformation):
    data = []
    idx = -1
    for file in tqdm(os.listdir(data_dir)):
        idx += 1
        file = data_dir + file
        img = cv2.imread(file)
        img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
        data.append(np.array(transformation(np.array(img))))
    np.random.shuffle(data)
    data = torch.from_numpy(np.array(data))
    print(idx)
    return data

In [5]:
data = load_data()

100%|██████████| 1081/1081 [00:10<00:00, 99.87it/s] 


1080


In [6]:
from torch import nn

In [7]:
class Desc(Module):
    def __init__(self,activation=LeakyReLU,starter=128):
        super().__init__()
        self.dropout = Dropout()
        self.activation = activation()
        self.linear1 = Linear(IMG_SIZE*IMG_SIZE*3,starter)
        self.linear1batchnorm = BatchNorm1d(starter)
        self.linear2 = Linear(starter,starter*2)
        self.linear2batchnorm = BatchNorm1d(starter*2)
        self.linear3 = Linear(starter*2,starter*4)
        self.linear3batchnorm = BatchNorm1d(starter*4)
        self.linear4 = Linear(starter*4,starter*8)
        self.linear4batchnorm = BatchNorm1d(starter*8)
        self.linear5 = Linear(starter*8,starter*4)
        self.linear5batchnorm = BatchNorm1d(starter*4)
        self.output = Linear(starter*4,1)
        self.output_activation = Sigmoid()
    
    def forward(self,X):
        X = X.view(-1,IMG_SIZE*IMG_SIZE*3)
        preds = self.dropout(self.activation(self.linear1batchnorm(self.linear1(X))))
        preds = self.dropout(self.activation(self.linear2batchnorm(self.linear2(preds))))
        preds = self.dropout(self.activation(self.linear3batchnorm(self.linear3(preds))))
        preds = self.dropout(self.activation(self.linear4batchnorm(self.linear4(preds))))
        preds = self.dropout(self.activation(self.linear5batchnorm(self.linear5(preds))))
        preds = self.output_activation(self.output(preds))
        return preds
        
class Gen(Module):
    def __init__(self,z_dim,activation=LeakyReLU,starter=512):
        super().__init__()
        self.dropout = Dropout()
        self.activation = activation()
        self.linear1 = Linear(z_dim,starter)
        self.linear1batchnorm = BatchNorm1d(starter)
        self.linear2 = Linear(starter,starter*2)
        self.linear2batchnorm = BatchNorm1d(starter*2)
        self.linear3 = Linear(starter*2,starter*4)
        self.linear3batchnorm = BatchNorm1d(starter*4)
        self.linear4 = Linear(starter*4,starter*8)
        self.linear4batchnorm = BatchNorm1d(starter*8)
        self.linear5 = Linear(starter*8,starter*4)
        self.linear5batchnorm = BatchNorm1d(starter*4)
        self.output = Linear(starter*4,IMG_SIZE*IMG_SIZE*3)
        self.output_activation = Tanh()
 
    def forward(self, X):
        preds = self.dropout(self.activation(self.linear1batchnorm(self.linear1(X))))
        preds = self.dropout(self.activation(self.linear2batchnorm(self.linear2(preds))))
        preds = self.dropout(self.activation(self.linear3batchnorm(self.linear3(preds))))
        preds = self.dropout(self.activation(self.linear4batchnorm(self.linear4(preds))))
        preds = self.dropout(self.activation(self.linear5batchnorm(self.linear5(preds))))
        preds = self.output_activation(self.output(preds))
        return preds

In [8]:
z_dim = 64
batch_size = 32
desc = Desc().to(device)
gen = Gen(z_dim).to(device)
lr = 3e-4
epochs = 62
criterion = BCELoss()
optimizer_gen = Adam(gen.parameters(),lr=lr)
optimizer_desc = Adam(desc.parameters(),lr=lr)
fixed_noise = torch.randn((batch_size,z_dim)).to(device)

In [9]:
data = data[0:1056]

In [10]:
def accuracy_fake(desc_fake):
    correct = 0
    total = 0
    preds = np.round(np.array(desc_fake.cpu().detach().numpy()))
    for pred in preds:
        if pred == 0:
            correct += 1
        total += 1
    return round(correct/total,3)
def accuracy_real(desc_real):
    correct = 0
    total = 0
    preds = np.round(np.array(desc_real.cpu().detach().numpy()))
    for pred in preds:
        if pred == 1:
            correct += 1
        total += 1
    return round(correct/total,3)

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline')
for epoch in tqdm(range(epochs)):
    for idx in range(0,len(data),batch_size):
#         try:
            data_batch = data[idx:idx+batch_size].view(-1,3,IMG_SIZE,IMG_SIZE).to(device).float()
            batch_size = data_batch.shape[0]
            noise = torch.randn(batch_size,z_dim).to(device)
            fake = gen(noise)
            fake.to(device)
            desc_fake = desc(fake).view(-1)
            lossD_fake = criterion(desc_fake,torch.zeros_like(desc_fake))
            desc_real = desc(data_batch).view(-1)
            lossD_real = criterion(desc_real,torch.ones_like(desc_real))
            lossD = (lossD_real+lossD_fake)/2
            optimizer_desc.zero_grad()
            lossD.backward(retain_graph=True)
            optimizer_desc.step()
            output = desc(fake).view(-1)
            lossG = criterion(output,torch.ones_like(output))
            optimizer_gen.zero_grad()
            lossG.backward()
            optimizer_gen.step()
            wandb.log({'lossD':lossD.item()})
            wandb.log({'lossG':lossG.item()})
            wandb.log({'lossD Fake':lossD_fake.item()})
            wandb.log({'lossD Real':lossD_real.item()})
            wandb.log({'Acc Fake':accuracy_fake(desc_fake)})
            wandb.log({'Acc Real':accuracy_real(desc_real)})
#         except Exception as e:
#             print(e)
    with torch.no_grad():
        fake = gen(fixed_noise).view(-1,3,IMG_SIZE,IMG_SIZE)
        img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
        wandb.log({'img':wandb.Image(img_grid_fake)})
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


 56%|█████▋    | 35/62 [03:42<02:55,  6.48s/it]

In [12]:
# transformations = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
# for transformation in transformations:
#     transformation = Compose([
#         ToTensor(),
#         Normalize(transformation,transformation)
#     ])
#     data = load_data(transformation=transformation)
#     wandb.init(project=PROJECT_NAME,name=f'transformation-{transformation}')
#     for epoch in tqdm(range(epochs)):
#         for idx in range(0,len(data),batch_size):
#     #         try:
#                 data_batch = data[idx:idx+batch_size].view(-1,3,IMG_SIZE,IMG_SIZE).to(device).float()
#                 batch_size = data_batch.shape[0]
#                 noise = torch.randn(batch_size,z_dim).to(device)
#                 fake = gen(noise)
#                 fake.to(device)
#                 desc_fake = desc(fake).view(-1)
#                 lossD_fake = criterion(desc_fake,torch.zeros_like(desc_fake))
#                 desc_real = desc(data_batch).view(-1)
#                 lossD_real = criterion(desc_real,torch.ones_like(desc_real))
#                 lossD = (lossD_real+lossD_fake)/2
#                 optimizer_desc.zero_grad()
#                 lossD.backward(retain_graph=True)
#                 optimizer_desc.step()
#                 output = desc(fake).view(-1)
#                 lossG = criterion(output,torch.ones_like(output))
#                 optimizer_gen.zero_grad()
#                 lossG.backward()
#                 optimizer_gen.step()
#                 wandb.log({'lossD':lossD.item()})
#                 wandb.log({'lossG':lossG.item()})
#                 wandb.log({'lossD Fake':lossD_fake.item()})
#                 wandb.log({'lossD Real':lossD_real.item()})
#                 wandb.log({'Acc Fake':accuracy_fake(desc_fake)})
#                 wandb.log({'Acc Real':accuracy_real(desc_real)})
#     #         except Exception as e:
#     #             print(e)
#         with torch.no_grad():
#             fake = gen(fixed_noise).view(-1,3,IMG_SIZE,IMG_SIZE)
#             img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
#             wandb.log({'img':wandb.Image(img_grid_fake)})
#     wandb.finish()